In [2]:
from spacepy import pycdf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns
from scipy import stats
from mpl_toolkits.mplot3d import axes3d
import matplotlib
import matplotlib.colors as colors
from collections import Counter
from scipy.optimize import curve_fit
from scipy import asarray as ar,exp
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
import math
from lmfit.models import GaussianModel
from sklearn import mixture
from matplotlib.colors import LogNorm
from sklearn import cluster
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile

# Maxwellian Fit Loop

In [101]:
year_ref = '2007'
bpfile = np.load(str(year_ref)+'_bp_180_strahl.npz')
all_bp = bpfile['arr_0']

npzfile = np.load(str(year_ref)+'.npz')

start_dc = npzfile['arr_0']
end_dc = npzfile['arr_1']
start_c = npzfile['arr_2']
end_c = npzfile['arr_3']

date2 = pd.Timestamp.date(start_dc[len(start_dc)-1])

core_t = []
core_n = []

for y in range(0,len(start_dc)):

    event = y
    #print(event)

    time = pd.Timestamp.time(start_dc[event])
    date = pd.Timestamp.date(start_dc[event])
    time1 = pd.Timestamp.time(end_dc[event])
    date1 = pd.Timestamp.date(end_dc[event])

    array = (date.year,date.month,date.day,time.hour,time.minute,time.second)
    array1 = (date1.year,date1.month,date1.day,time1.hour,time1.minute,time1.second)

    strs = ["" for x in range(6)]
    strs1 = ["" for x in range(6)]

    for i in range(len(array)):
        if array[i]<10:
            strs[i]='0'+str(array[i])
        else:
            strs[i]=str(array[i])
        if array1[i]<10:
            strs1[i]='0'+str(array1[i])
        else:
            strs1[i]=str(array1[i])

    stamp = strs[0]+strs[1]+strs[2]+'_'+strs[3]+strs[4]+strs[5]+'_'+strs1[0]+strs1[1]+strs1[2]+'_'+strs1[3]+strs1[4]+strs1[5]

    choose_file = '/Users/mayur/PhD/Cluster_data/Statistical_Study/PSD/PSD_DC/C2_CP_PEA_PITCH_SPIN_PSD__'+stamp+'_V170624.cdf'

    #print(choose_file)

    cdf = pycdf.CDF(choose_file)
    #for (i, item) in enumerate(cdf, start=0):
    #    print(i, item, cdf[i])
    
    epoch = cdf[0][...]

    #data_raw = cdf[26][...] #DEFlux
    data = cdf[26][...] #PSD
    
    energy = cdf[20][...]
    
    if len(energy) < 45:
        print(' ')
        core_t.append(0)
        core_n.append(0)
        continue
    
    if energy[0,0] < 0:
        print(' ')
        core_t.append(0)
        core_n.append(0)
        continue
    
    ### Only reshape data for DEFlux, not PSD 
    #data = []
        
    #for f in range(len(epoch)):
    #    a = data_raw[f].flatten()
    #    b = a.reshape(44,12,order='F')
    #    c = b.flatten()
    #    d = c.reshape(12,44)
    #    data.append(d)
    
    #data = np.array(data)
    
    pa = cdf[23][...]
    
    if all_bp[event] > 90:
        print(' ')
        core_t.append(0)
        core_n.append(0)
        continue
        
    if all_bp[event] < 20:
        print(' ')
        core_t.append(0)
        core_n.append(0)
        continue
    
    try:
        ma = np.asscalar(np.where(energy[0]==all_bp[event])[0]) #start of energy range Maxwell fit (higher energy)
    except:
        print(' ')
        core_t.append(0)
        core_n.append(0)
        continue
    
    mb = np.asscalar(np.where(energy[0]==10.073)[0]) #end of energy range Maxwell fit (lowest energy)
    fa = np.asscalar(np.where(energy[0]==173.09)[0]) #start of energy range bp analysis

    yt_90 = []
    
    for n in range(ma,mb):
        y_90 = data[0:len(epoch),11,n]
        ym_90 = np.mean(y_90)
        yt_90.append(ym_90)


    x = np.sqrt((energy[0,ma:mb]))
    y = (yt_90)
    
    m = 1
    k = 1

    n_scale = np.max(y)
    T_scale = 1
    T_func = 1

    def maxwellian(x,T_c,n_c,T_f):
        return n_c*(1/T_f)*((m/(2*np.pi*k))**(3/2))*np.exp(-m*((x)**2)/(2*k*T_c))

    popt,pcov = curve_fit(maxwellian,x,y,p0=[T_scale,n_scale,T_func],maxfev=10000)
    
    print(popt[0]) #[core temp, core density, function(core temp)]
    
    core_t.append(popt[0]) 
    core_n.append(popt[1])
    
    del yt_90

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
2.2398359943689696
6.194826122621784
6.639139723228137
7.459601388473479
10.023421794494315
10.119460777694592
8.246373586505692
10.178713094438388
7.324447412034546
5.731878356403128
6.621000426249185
6.859551094026129
 
6.236421061614819
7.004773609190311
5.3000655291901335
6.1937293398560405
11.505113312425369
8.930685961572571
 
4.7085192678120045
5.9047531494155825
3.194746965183225
4.069205267452198
3.8272300131560684
9.933945885784713
2.4130528682035046
6.2045413310532584
3.3521248193223574
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
6.16875743324764
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
5.4268986965902135
7.050240286292827
6.686068633495949
7.487094800461384
6.967306575065161
6.6679470083012085
6.883670463867818
7.289589